# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [2]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [3]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [4]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [5]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [6]:
def foldl2(f, x0, lst):
    return foldr(lambda ev, F: lambda val: F(f(val, ev)), lambda x: x, lst)(x0)
foldl2(f, 1, [1, 2, 3])

0.16666666666666666

In [7]:
def foldr2(f, x0, lst):
    return foldl(lambda F, ev: lambda val: F(f(ev, val)), lambda x: x, lst)(x0)
foldr2(f, 1, [1, 2, 3])

1.5

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [1]:
import string


def check_inv(a, b):
    if len(a) < len(b):
        return False
    b_chars = dict.fromkeys(string.ascii_letters, 0)
    for i in b:
        b_chars[i] += 1
    a_chars = dict.fromkeys(string.ascii_letters, 0)
    for i in a:
        a_chars[i] += 1
    for i in b:
        if a_chars[i] < b_chars[i]:
            return False
    return True


check_inv('abcrotm', 'tro')

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [12]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value

    def __iter__(self):
        if self.left is None and self.right is None:  # так как итерация по листьям
            yield self.value
        if self.left:
            yield from self.left.__iter__()
        if self.right:
            yield from self.right.__iter__()

    def __str__(self):
        s = str(self.value) + ' '
        if self.left:
            s += (self.left.__str__())
        if self.right:
            s += (self.right.__str__())
        return s

    def __repr__(self):
        if self.left is None and self.right is None:
            return f'Leaf({self.value})'
        return f'Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})'
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

print(list(tree) == [3, 4, 2])
print(tree)
print(repr(tree))

True
0 1 3 4 2 
Tree(0, Tree(1, Leaf(3), Leaf(4)), Leaf(2))


# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [11]:
oper = {'*': lambda a, b: a * b, '/': lambda a, b: b / a, '+': lambda a, b: a + b, '-': lambda a, b: b - a}
prior = {1: ['+', '-'], 2: ['*', '/'], 0: ['(', ')']}


def priority(val):
    for k, v in prior.items():
        if val in v:
            return k
    return -1


def split(expr):
    s = []
    for i in expr.split():
        if len(i) > 1:
            if i[0] == '(':
                s.append(i[0])
                s.append(i[1:])
            elif i[-1] == ')':
                s.append(i[:-1])
                s.append(i[-1])
            else:
                s.append(i)
        else:
            s.append(i)
    return s


def calc(expr):
    s = split(expr)
    #print(s)
    result = []
    stack = []
    for el in s:
        #print(result)
        #print(stack)
        if el not in '+-*/()':
            result.append(int(el))
        else:
            if el == '(':
                stack.append(el)
                continue
            elif el == ')':
                while stack[-1] != '(':
                    result.append(oper[stack.pop()](result.pop(), result.pop()))
                stack.pop()
            else:
                if len(stack) == 0:
                    last = None
                else:
                    last = stack[-1]
                while priority(last) >= priority(el):
                    result.append(oper[stack.pop()](result.pop(), result.pop()))
                    if len(stack) == 0:
                        last = None
                    else:
                        last = stack[-1]
                stack.append(el)
    
    while len(stack) > 0:
        result.append(oper[stack.pop()](result.pop(), result.pop()))

    return result[0]

print(calc('2 * (15 - 3 * 4) - 2') == 4)
print(calc('3 * 4 / (2 * 2) * 2 * 3'))

True
18.0
